In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error , mean_absolute_error , mean_absolute_percentage_error
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import LSTM , Dense , Dropout , GRU , Concatenate , Input , Conv1D , InputLayer
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
import math

<Strong> Grabbing the data from stored files (originally pulled from Binance) </Strong>

In [3]:
df_btc = pd.read_csv("BTC_1h_data.csv")

In [4]:
df_btc

date      open      high       low     close  \
0      2017-08-17 04:00:00.000   4261.48   4313.62   4261.32   4308.83   
1      2017-08-17 05:00:00.000   4308.83   4328.69   4291.37   4315.32   
2      2017-08-17 06:00:00.000   4330.29   4345.45   4309.37   4324.35   
3      2017-08-17 07:00:00.000   4316.62   4349.99   4287.41   4349.99   
4      2017-08-17 08:00:00.000   4333.32   4377.85   4333.32   4360.69   
...                        ...       ...       ...       ...       ...   
39216      2022-02-11 10:00:00  43492.63  43569.67  43233.67  43443.77   
39217      2022-02-11 11:00:00  43443.78  43523.42  43320.00  43357.53   
39218      2022-02-11 12:00:00  43357.53  43700.78  43253.31  43570.77   
39219      2022-02-11 13:00:00  43570.77  43826.00  43503.01  43765.14   
39220      2022-02-11 14:00:00  43765.15  43814.17  43450.01  43534.54   

            volume  
0        47.181009  
1        23.234916  
2         7.229691  
3         4.443249  
4         0.972807  
...            ...  
39216  1305.626920  
39217   744.524180  
39218  1317.546940  
39219  1564.251800  
39220   880.790640  

[39221 rows x 6 columns]

In [5]:
df_headers = df_btc.columns

<Strong> Grabbing the closing price (univariate) </Strong>

In [6]:
# Removing everything but the closing price
btc_data = df_btc.values[:, 4 ,].astype(float)

btc_data[-1]

43534.54

<Strong> Scaling the data  </Strong>

In [8]:
percTrain = 70
percVal = 20 

In [9]:
scaler = MinMaxScaler()
    
onePercent = len(btc_data) // 100
numberTraining = onePercent * percTrain

reshaped_data = btc_data.reshape(-1,1)

#Just scaling on training data otherwise it would be leakage
scaler.fit(reshaped_data[:numberTraining])
scaled_btc = scaler.transform(reshaped_data)

<Strong> Creating Matrix in Sliding window form <Strong>

In [10]:
def sliding_window(elements, window_size):
    
    data = [] 
    targets = []
    
    if len(elements) <= window_size:
        return elements
    
    for i in range(len(elements) - window_size ):
        
        data.append(elements[i:i+window_size])
        targets.append(elements[i+window_size])
        
    return np.array(data) , np.array(targets)

In [11]:
# Using 24 datapoints to predict the 25th

window_length = 24
features = 1

sliding_winda_btc = sliding_window(scaled_btc , window_length)

<Strong> Splitting the data after we create Sliding Window matrix (more data) </Strong>

In [12]:
# Splitting the data after creating the sliding window data
def splitting_train_test(data):
        
    onePercent = len(data[1]) // 100
    
    numberTraining = onePercent * percTrain
    numberValidation = onePercent * percVal
    
    trainingData = data[0][:numberTraining] , data[1][:numberTraining]
    validationData = data[0][numberTraining : numberTraining + numberValidation] , data[1][numberTraining : numberTraining + numberValidation]
    testData = data[0][numberTraining + numberValidation:] , data[1][numberTraining + numberValidation:] 
    
    #Returning tuples of (sliding-window , target_values)
    return trainingData , validationData , testData

In [13]:
#Reshaping the data so we can use min-max a
btc_train , btc_val , btc_test = splitting_train_test(sliding_winda_btc)

print(btc_train[0].shape)

(27370, 24, 1)


In [14]:
btc_test[1].shape

(4007, 1)

<Strong> Creating the model From the parameters found in the tuner. </Strong>

In [15]:
inputs = Input(shape = (window_length , features) )

x = LSTM(30 , return_sequences = True )(inputs)
x = Dropout(0.01)(x) 
x = LSTM(50)(x)
x = Dense(128)(x)

y = GRU(30,input_shape=(window_length , features))(inputs)
y = Dropout(0.01)(y)
y = Dense(128)(y)

final = Concatenate()([x,y])
final = Dense(1)(final)

model = tf.keras.Model(inputs=inputs, outputs= final)

model.summary()
plot_model(model)

2022-04-12 12:24:46.216258: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24, 1)]      0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 24, 30)       3840        ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 24, 30)       0           ['lstm[0][0]']                   
                                                                                                  
 gru (GRU)                      (None, 30)           2970        ['input_1[0][0]']                
                                                                                              

<Strong> Creating a callback , so we can avail of early stopping </Strong>

In [16]:
earlyStopping = EarlyStopping(monitor = 'val_loss' , patience = 30 , mode = 'min' , verbose = 1)

In [17]:
#Validation set needs to be in a tuple with x , y

model.compile(optimizer='adam' , loss = 'mse')

<Strong> Fitting the Model , and storing the epoch training stopped on </Strong>

In [19]:
history = model.fit(btc_train[0] , btc_train[1] , validation_data = btc_val  , batch_size = 512  , epochs =300 , verbose = 1 , callbacks=[earlyStopping])
num_epochs = earlyStopping.stopped_epoch

Epoch 1/300


KeyboardInterrupt: 

In [20]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
#plt.ylim([0,0.01])
plt.show()

NameError: name 'history' is not defined

<Strong> Testing our model on the validation data once again , hyper-parameter adjustment. <Strong>

In [21]:
validation_predict = model.predict(btc_val[0])
validation_predict = scaler.inverse_transform(validation_predict)

validation_actual = scaler.inverse_transform(btc_val[1])
mean_absolute_error(validation_actual , validation_predict)

29474.34029096942

<Strong> Combining the train and validation set when perfected hyper-parameters and training on fresh model. </Strong>

In [ ]:
train_and_val = np.concatenate([btc_train[0] , btc_val[0]] )
train_and_val_targets = np.concatenate([btc_train[1] , btc_val[1] ])

freshModel = tf.keras.Model(inputs=inputs, outputs= final)

freshModel.compile(optimizer=opt , loss = 'mse')
final = freshModel.fit(train_and_val , train_and_val_targets , batch_size = 512  , epochs = num_epochs )

<Strong> Finally testing on test set and grabbing evaluation metrics - 1 day </Strong>

In [23]:
test_predict = freshModel.predict(btc_test[0])
test_actual = btc_test[1]

NameError: name 'freshModel' is not defined

In [ ]:
print(mean_absolute_error(test_actual , test_predict))
print(mean_absolute_percentage_error(test_actual , test_predict))
print(mean_squared_error(test_actual , test_predict , squared = False))
print(mean_squared_error(test_actual , test_predict))

<Strong> For Predicting more than 1 day in the future - Test Set. </Strong>

In [24]:
def steps_in_future(hours_in_future , data):
    
    #All the hours_in_future time predictions
    predictions = []
    
    # Have to cut off the (hours_in_future - 1) off the test set to avoid out of bounds error
    test_data = data[0][:-(hours_in_future - 1)]
    
    for x in range (len(test_data)):
        #Going through all the windows
        last_window = test_data[x].reshape(1,-1)
    
            # Make as many predictions as hours_in_future
        for i in range(hours_in_future):
            
            # Take the predicted value from the last window in training set
            last_prediction = model.predict(last_window)[0]

            #shifting the window size one step down
            last_window[0] = np.roll(last_window[0], -1)

            #replacing the old value with new prediction
            last_window[0 , (len(last_window[0]) - 1)] = last_prediction

        #append prediction
        predictions.append(last_prediction)
    
    return predictions

In [ ]:
three_day_predictions = steps_in_future(3, btc_test)
five_day_predictions = steps_in_future(5, btc_test)
seven_day_predictions = steps_in_future( 7, btc_test)
nine_day_predictions = steps_in_future( 9, btc_test)
eleven_day_predictions = steps_in_future( 11, btc_test)

In [ ]:
# Targets to compare  

targets_in_future_three = btc_test[1][(3 - 1):]
targets_in_future_five = btc_test[1][(5 - 1):]
targets_in_future_seven = btc_test[1][(7 - 1):]
targets_in_future_nine = btc_test[1][(9 - 1):]
targets_in_future_eleven = btc_test[1][(11 - 1):]

In [ ]:
def get_eval_metrics(num_days , actual , results):
    
    mae = mean_absolute_error(actual , results)
    mse = mean_squared_error(actual , results )
    rmse = mean_squared_error(actual , results , squared = False)
    mape = mean_absolute_percentage_error(actual , results)

    print(num_days , "MAE :" , mae ,"MSE :" , mse , "RMSE :" , rmse , "MAPE :" , mape)

    return mae , mse , rmse , mape

In [ ]:
three_days_mae , three_days_mse , three_days_rmse , three_days_mape = get_eval_metrics("Three Hour Forecast" , targets_in_future_three , three_day_predictions)
five_days_mae , five_days_mse , five_days_rmse , five_days_mape = get_eval_metrics("Five Hour Forecast" , targets_in_future_five , five_day_predictions)
seven_days_mae , seven_days_mse , seven_days_rmse , seven_days_mape = get_eval_metrics("Seven Hour Forecast" , targets_in_future_seven , seven_day_predictions)
nine_days_mae , nine_days_mse , nine_days_rmse , nine_days_mape = get_eval_metrics("Nine Hour Forecast" , targets_in_future_nine , nine_day_predictions)
eleven_days_mae , eleven_days_mse , eleven_days_rmse , eleven_days_mape = get_eval_metrics("Eleven Hour Forecast" , targets_in_future_eleven , eleven_day_predictions)

